reference: https://www.tensorflow.org/tutorials/quickstart/advanced?hl=ko

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, BatchNormalization, Dense, Activation, add, Flatten, GlobalAveragePooling2D, Reshape, multiply
from tensorflow.keras import Model
from tensorflow.keras import utils

In [ ]:

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0



170508288/170498071 [==============================] - 2s 0us/step


In [ ]:

num_classes = 10
y_train = utils.to_categorical(y_train)
y_test = utils.to_categorical(y_test)

In [ ]:
data_augmentation = tf.keras.Sequential(
    [
        tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal"),
        tf.keras.layers.experimental.preprocessing.RandomRotation(0.1),
        tf.keras.layers.experimental.preprocessing.RandomZoom(0.1),
    ]
)

In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)

train_ds = train_ds.map(lambda x, y: (data_augmentation(x), y))

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [ ]:
def Residual_Block_50(x, filters):
    f1, f2 = filters
    x_skip = x 

    x = Conv2D(f1, kernel_size=(1,1), strides=1, padding='same')(x) 
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(f1, kernel_size=(3,3), strides=1, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(f2, kernel_size=(1,1), strides=1, padding='same')(x) 
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x_skip = Conv2D(f2, kernel_size=(1,1),strides=(1,1),padding='same')(x_skip)
    x_skip = BatchNormalization()(x_skip)  

    x = add([x, x_skip])
    x = Activation('relu')(x)    
    return x

In [ ]:
input_shape = x_train[0].shape    

input_layer  = Input(shape=input_shape)

x = Conv2D(64, kernel_size=7, strides=2, padding='valid')(input_layer)
x = MaxPooling2D(pool_size=(3,3), strides=2, padding='same')(x)

for i in range(3):
  x = Residual_Block_50(x, filters=(64,256)) 

for i in range(4):
  x = Residual_Block_50(x, filters =(128,512)) 

for i in range(6):
  x = Residual_Block_50(x, filters =(256,1024)) 

for i in range(3):
  x = Residual_Block_50(x, filters =(512,2048)) 

x = GlobalAveragePooling2D()(x) 

output_layer = Dense(10, activation = 'softmax')(x)




In [ ]:
resNet = tf.keras.Model(input_layer, output_layer)
resNet.summary()

loss object 및 optimizer 정의: categorical cross entropy, SGD

In [ ]:
loss_object =

optimizer = 

모델의 손실과 성능을 측정할 지표 

In [ ]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.CategoricalAccuracy(name='test_accuracy')

`tf.GradientTape`를 사용하여 모델 훈련을 위한 함수 train step 구현

In [ ]:
@tf.function
def train_step(images, labels):
 

test step 함수 구현

In [ ]:
@tf.function
def test_step(images, labels):
 

training / validation

In [ ]:
EPOCHS = 10

train_acc_list=[]
train_loss_list=[]
test_acc_list=[]
test_loss_list=[]

for epoch in range(EPOCHS):
  cur_step = 0


  ## train
  ## test


  train_acc_list.append(train_accuracy.result())
  train_loss_list.append(train_loss.result())
  test_acc_list.append(test_accuracy.result())
  test_loss_list.append(test_loss.result())

  template = '에포크: {}, 손실: {}, 정확도: {}, 테스트 손실: {}, 테스트 정확도: {}'
  print (template.format(epoch+1,
                         train_loss.result(),
                         train_accuracy.result()*100,
                         test_loss.result(),
                         test_accuracy.result()*100))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
epochs_range = range(10)
len(np.array(train_loss_list))

In [ ]:
plt.figure(figsize=(8, 8))
plt.plot(epochs_range, np.array(test_acc_list), label='Test Accuracy')

In [ ]:
plt.figure(figsize=(8, 8))
#plt.subplot(1, 2, 1)
plt.plot(epochs_range, np.array(train_loss_list), label='Training Loss')